# Intro

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sacc
import pyccl as ccl
import astropy.table as tbl
import os
import emcee
from scipy.optimize import minimize
import corner
import pickle

In [2]:
nzbins = 12 # Number of galaxy redshift bins
nls = 7 # Number of cl data points

rker_z_min = 0.002 # Minimum redshift for neutrino radial kernel
rker_z_max = 4 # Maximum redshift for neutrino radial kernel
rker_z_n = 1024 # Number of redshift/radial distance values in neutrino radial kernel
gal_z_dir = ['../GalZDist', '../GalZDist_2']  # Directory containing redshift distributions of each galaxy map
bzs = [1.182, 1.086, 1.126, 1.144, 1.206, 1.548, 
       1.13, 1.40, 1.35, 1.77, 2.1, 2.5] # Bias values for each galaxy map

alpha_val = -1 # Fixed alpha value
write_to_file = True # Whether to write results to file 'A_constraints.txt'

# xCell Power Spectra (Data)

In [3]:
# Loads in data, only keeps first 7 data points to match previous calculations
def load_data(path):
    s = sacc.Sacc.load_fits(path)
    s.remove_selection(ell__gt=352)
    return s

In [4]:
def get_sacc_info(s, include_first_ell):
    
    # Finds relevant indices in sacc object
    indices = []
    for i in range(nzbins):
        ind_here = s.indices('cl_00', (f'LOWZ__{i}', 'IceCubeY10'))
        if include_first_ell:
            indices.append(list(ind_here))
        else:
            indices.append(list(ind_here)[1:])
    indices = np.array(indices)

    # Gets ells, cls, covs, and inverse covs
    ells, _ = s.get_ell_cl('cl_00', 'LOWZ__0', 'IceCubeY10')
    if not include_first_ell:
        ells = ells[1:]
    cov_total = s.covariance.covmat
    cls = []
    covs = []
    icovs = []
    for i in range(nzbins):
        ind = indices[i]
        cls.append(s.mean[ind])
        covs.append(np.array(cov_total[ind][:, ind]))
        icovs.append(np.array(np.linalg.inv(covs[i])))
    cls = np.array(cls)
    covs = np.array(covs)
    icovs = np.array(icovs)

    # Finds relevant indices in sacc object
    indices = []
    for i in range(nzbins):
        ind_here = s.indices('cl_00', (f'LOWZ__{i}', 'IceCubeY10'))
        if include_first_ell:
            indices += list(ind_here)
        else:
            indices += list(ind_here)[1:]
    indices = np.array(indices)

    # Finds quantities for combined bin
    cls_comb = np.array(s.mean[indices])
    cov_total = s.covariance.covmat
    covs_comb = np.array(cov_total[indices][:, indices])
    icovs_comb = np.array(np.linalg.inv(covs_comb))
    return ells, cls_comb, icovs_comb, cls, icovs

# Theoretical Power Spectra (Model)

In [5]:
# Sets cosmological model
cosmo = ccl.CosmologyVanillaLCDM(transfer_function='eisenstein_hu')

In [6]:
# Loads in galaxy redshift distributions into astropy tables
gal_z_filenames = sorted(os.listdir(gal_z_dir[0]))

gal_z_dist = []
for i in gal_z_filenames:
    gal_z_dist.append(tbl.Table.read(gal_z_dir[0] + '/' + i, format = 'ascii'))

[gal_z_dist.append(tbl.Table.read(gal_z_dir[1]+f'/dndz_DELS__{i}_nuX.txt', 
                                  format = 'ascii')) for i in range(4)]
[gal_z_dist.append(tbl.Table.read(gal_z_dir[1]+f'/dndz_SDSS__QSO{i}_nuX.txt', 
                                  format = 'ascii')) for i in range(2)]

[None, None]

In [7]:
# Creates tracer for each galaxy redshift bin
gal_tracers = []
for i in range(nzbins):
    z = np.array(gal_z_dist[i]['col1'])
    nz = np.array(gal_z_dist[i]['col2'])
    bz = np.full(len(z), bzs[i])
    gal_tracers.append(ccl.NumberCountsTracer(cosmo, has_rsd=False, 
                                              dndz=(z, nz), bias=(z, bz)))

In [8]:
# Finds array of radial distances (chi) of radial kernel
chi_min = ccl.comoving_radial_distance(cosmo, 1./(1+rker_z_min))
chi_max = ccl.comoving_radial_distance(cosmo, 1./(1+rker_z_max))
chi_arr = np.linspace(chi_min, chi_max, rker_z_n)

# Converts radial distance array to inverse scale factor array (i.e. 1+z)
inv_a_arr = 1/ccl.scale_factor_of_chi(cosmo, chi_arr)

In [9]:
# Generates neutrino tracer
def generate_nu_tracer(cosmo, A, alpha=alpha_val, 
                       inv_a_arr=inv_a_arr):
    # cosmo - cosmological model to use
    # A - constant of proportionality in neutrino radial kernel
    # alpha - index of power in neutrino radial kernel
    # z_min - minimum redshift for neutrino radial kernel
    # z_max - maximum redshift for neutrino radial kernel
    # nchi - number of redshift/radial distance values in neutrino radial kernel

    # Uses chosen A and alpha to find final array of radial kernel
    rker_arr = A*(inv_a_arr)**alpha

    # Adds radial kernel to tracer
    nu_tracer = ccl.Tracer()
    nu_tracer.add_tracer(cosmo, kernel = (chi_arr, rker_arr))
    return nu_tracer

In [10]:
# Finds theoretical/model power spectra using neutrino radial kernel of the form A*(1+z)^alpha
def model(ells, A, zbin, alpha=alpha_val):
    # A - constant of proportionality in neutrino radial kernel
    # zbin - which redshift bin to use
    # alpha - index of power in neutrino radial kernel
    
    nu_tracer = generate_nu_tracer(cosmo, A, alpha)
    if zbin == 'all':
        cl_cross = []
        for i in gal_tracers:
            cl_cross += list(ccl.angular_cl(cosmo, nu_tracer, i, ells))
    else:
        cl_cross = ccl.angular_cl(cosmo, nu_tracer, gal_tracers[zbin], ells)
    return np.array(cl_cross)

# Parameter Inference

In [11]:
# Finds mean and standard deviation
def find_mean_std(ells, icov, d, zbin, alpha=alpha_val):
    t = model(ells, 1, zbin, alpha)
    tt = np.dot(t, np.dot(icov, t))
    td = np.dot(t, np.dot(icov, d))
    return td/tt, np.sqrt(1/tt)

In [12]:
def find_all_mean_std(ells, cls_comb, icovs_comb, cls, icovs):

    # Finds mean and std of combined bin
    A_mean_comb, A_std_comb = find_mean_std(ells, icovs_comb, cls_comb, 'all')

    # Finds mean and std of each split bin
    A_mean = np.empty(nzbins)
    A_std = np.empty(nzbins)
    for i in range(nzbins):
        A_mean[i], A_std[i] = find_mean_std(ells, icovs[i], cls[i], i)
    return A_mean_comb, A_std_comb, A_mean, A_std

In [13]:
# Writes results to output file
def write_results(flux, include_first_ell, A_mean_comb, A_std_comb, A_mean, A_std):
    with open('A_Constraints.txt', 'a+') as fp:
        fp.write('\n')
        fp.write('\n')
        fp.write(flux+' '+str(include_first_ell)+' all '+str(A_mean_comb)+' '+str(A_std_comb))
    for i in range(nzbins):
        with open('A_Constraints.txt', 'a+') as fp:
            fp.write('\n')
            fp.write('\n')
            fp.write(flux+' '+str(include_first_ell)+' '+str(i)+' '+str(A_mean[i])+' '+str(A_std[i]))

In [14]:
# Gets constraints
def get_constraints(flux, include_first_ell, write_to_file=write_to_file):
    path = f'cl_flux_{flux}.fits'
    s = load_data(path)
    ells, cls_comb, icovs_comb, cls, icovs = get_sacc_info(s, include_first_ell)
    A_mean_comb, A_std_comb, A_mean, A_std = find_all_mean_std(ells, cls_comb, icovs_comb, cls, icovs)
    if write_to_file:
        write_results(flux, include_first_ell, A_mean_comb, A_std_comb, A_mean, A_std)

In [15]:
if write_to_file:
    # Deletes existing file
    if os.path.exists('A_Constraints.txt'):
        os.remove('A_Constraints.txt')

    # Creates new file
    with open('A_Constraints.txt', 'w') as fp:
        fp.write('# Flux, l2, bin, A_mean, A_std')
    
get_constraints('number', True)
get_constraints('number', False)
get_constraints('energy', True)
get_constraints('energy', False)